<!-- ---
title: "Momentum in Optimization"
format: 
  revealjs:
    theme: custom.scss
    css: custom.css
    width: 1920
    height: 1080
    menu:
      side: right
      width: wide
    template-partials:
      - title-slide.html
    slide-number: c/t
    logo: "eclipse_logo_small.png"
    highlight-style: a11y
    incremental: false
    background-transition: fade
    footer: "©Philipp Pelz - FAU Erlangen-Nürnberg - Data Science for Electron Microscopy"
execute:
  eval: true
  echo: true
--- -->

# Momentum in Optimization

- Momentum is a key optimization technique in deep learning
- Addresses challenges in stochastic gradient descent:
  - Learning rate sensitivity
  - Convergence issues
  - Noise handling
- Particularly effective for ill-conditioned problems

## Basics of Momentum

### Leaky Averages

- Minibatch SGD averages gradients to reduce variance
- Momentum extends this concept using "leaky averages":
  - Accumulates past gradients
  - Weights recent gradients more heavily
  - Formula: $\mathbf{v}_t = \beta \mathbf{v}_{t-1} + \mathbf{g}_{t, t-1}$
  - $\beta \in (0, 1)$ controls the "memory" of past gradients

---


### Key Benefits

- Accelerates convergence
- Particularly effective for:
  - Ill-conditioned problems
  - Narrow canyons in optimization landscape
- Provides more stable descent directions
- Works well with both:
  - Noise-free convex problems
  - Stochastic gradient descent

## Visualizing the Problem

Let's examine an ill-conditioned problem:


In [ ]:
#| label: setup
%matplotlib inline
import d2l
import torch

eta = 0.4
def f_2d(x1, x2):
    return 0.1 * x1 ** 2 + 2 * x2 ** 2
def gd_2d(x1, x2, s1, s2):
    return (x1 - eta * 0.2 * x1, x2 - eta * 4 * x2, 0, 0)

d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))

## The Challenge

- Function $f(\mathbf{x}) = 0.1 x_1^2 + 2 x_2^2$ is very flat in $x_1$ direction
- Gradient in $x_2$ direction:
  - Much higher
  - Changes more rapidly
- Trade-off in learning rate:
  - Small rate: Slow convergence in $x_1$
  - Large rate: Divergence in $x_2$

## Momentum Method

### Update Equations

$$
\begin{aligned}
\mathbf{v}_t &\leftarrow \beta \mathbf{v}_{t-1} + \mathbf{g}_{t, t-1}, \\
\mathbf{x}_t &\leftarrow \mathbf{x}_{t-1} - \eta_t \mathbf{v}_t.
\end{aligned}
$$

### Implementation


In [ ]:
#| label: momentum-implementation
def momentum_2d(x1, x2, v1, v2):
    v1 = beta * v1 + 0.2 * x1
    v2 = beta * v2 + 4 * x2
    return x1 - eta * v1, x2 - eta * v2, v1, v2

eta, beta = 0.6, 0.5
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

## Effect of Momentum Parameter

- $\beta = 0.5$: Good convergence
- $\beta = 0.25$: Barely converges but better than no momentum
- $\beta = 0$: Reduces to regular gradient descent


In [ ]:
#| label: momentum-beta
eta, beta = 0.6, 0.25
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

## Effective Sample Weight

- Sum of weights: $\sum_{\tau=0}^\infty \beta^\tau = \frac{1}{1-\beta}$
- Step size effectively becomes $\frac{\eta}{1-\beta}$
- Better behaved descent direction


In [ ]:
#| label: effective-weight
d2l.set_figsize()
betas = [0.95, 0.9, 0.6, 0]
for beta in betas:
    x = d2l.numpy(d2l.arange(40))
    d2l.plt.plot(x, beta ** x, label=f'beta = {beta:.2f}')
d2l.plt.xlabel('time')
d2l.plt.legend();

## Practical Implementation

### From Scratch


In [ ]:
#| label: momentum-scratch
def init_momentum_states(feature_dim):
    v_w = d2l.zeros((feature_dim, 1))
    v_b = d2l.zeros(1)
    return (v_w, v_b)

def sgd_momentum(params, states, hyperparams):
    for p, v in zip(params, states):
        with torch.no_grad():
            v[:] = hyperparams['momentum'] * v + p.grad
            p[:] -= hyperparams['lr'] * v
        p.grad.data.zero_()

### Training with Different Parameters


In [ ]:
#| label: momentum-training
def train_momentum(lr, momentum, num_epochs=2):
    d2l.train_ch11(sgd_momentum, init_momentum_states(feature_dim),
                   {'lr': lr, 'momentum': momentum}, data_iter,
                   feature_dim, num_epochs)

data_iter, feature_dim = d2l.get_data_ch11(batch_size=10)
train_momentum(0.02, 0.5)

## Theoretical Analysis

### Quadratic Convex Functions

- General form: $h(\mathbf{x}) = \frac{1}{2} \mathbf{x}^\top \mathbf{Q} \mathbf{x} + \mathbf{x}^\top \mathbf{c} + b$
- For positive definite $\mathbf{Q}$:
  - Minimizer at $\mathbf{x}^* = -\mathbf{Q}^{-1} \mathbf{c}$
  - Minimum value: $b - \frac{1}{2} \mathbf{c}^\top \mathbf{Q}^{-1} \mathbf{c}$
- Gradient: $\partial_{\mathbf{x}} h(\mathbf{x}) = \mathbf{Q} (\mathbf{x} - \mathbf{Q}^{-1} \mathbf{c})$

---

### Convergence Analysis

- For scalar function $f(x) = \frac{\lambda}{2} x^2$:
  - Gradient descent: $x_{t+1} = (1 - \eta \lambda) x_t$
  - Convergence when $|1 - \eta \lambda| < 1$
  - Exponential convergence rate


In [ ]:
#| label: convergence-analysis
lambdas = [0.1, 1, 10, 19]
eta = 0.1
d2l.set_figsize((6, 4))
for lam in lambdas:
    t = d2l.numpy(d2l.arange(20))
    d2l.plt.plot(t, (1 - eta * lam) ** t, label=f'lambda = {lam:.2f}')
d2l.plt.xlabel('time')
d2l.plt.legend();

## Summary

- Momentum replaces gradients with leaky averages
- Key benefits:
  - Accelerates convergence
  - Works for both noise-free and noisy gradients
  - Prevents optimization stalling
  - Effective sample size: $\frac{1}{1-\beta}$
- Implementation requires:
  - Additional state vector (velocity)
  - Careful parameter tuning

## Exercises

1. Experiment with different momentum and learning rate combinations
2. Analyze gradient descent and momentum for quadratic problems with multiple eigenvalues
3. Derive minimum value and minimizer for $h(\mathbf{x}) = \frac{1}{2} \mathbf{x}^\top \mathbf{Q} \mathbf{x} + \mathbf{x}^\top \mathbf{c} + b$
4. Investigate behavior with stochastic gradient descent and minibatch variants 